# Sistema Multi-Robôs para Cobertura Eficiente de Ambientes Domésticos

- Bernardo Nogueira Borges
- Daniele Cássia Silva Diniz

Uma aplicação relevante na área de robótica é a dos robôs aspiradores, que têm a
função de percorrer o ambiente doméstico de forma autônoma, recolhendo detritos
ao longo do caminho. Este projeto abordará o problema de cobertura no contexto de
sistemas multi-robôs, com o objetivo de dividir a tarefa entre dois robôs. Um dos
principais desafios nesta área é garantir que os robôs executem seu trajeto de forma
eficiente, evitando a movimentação aleatória pelo espaço, como observado em
robôs aspiradores convencionais, conforme demonstrado no [vídeo](https://www.youtube.com/watch?v=qHEJhJ_CuOQ) da Neato
Robotics

Para superar esses desafios, utilizaremos nosso conhecimento em mapeamento,
controle e algoritmos para desenvolver um sistema de multi-robôs que abordará o
problema de **cobertura** do ambiente de forma eficaz, completa e, adicionalmente,
enfrentaremos o desafio de implementar a colaboração entre os robôs. A
implementação será realizada utilizando a linguagem **Python** e o simulador
**CoppeliaSim**.

Abordar este problema é crucial para aumentar a eficiência energética e reduzir a
pegada de carbono dos robôs aspiradores domésticos. Um planejamento adequado
permite que esses robôs executem suas tarefas de maneira mais rápida e eficiente,
contribuindo significativamente para a sustentabilidade ambiental. As principais
aplicações incluem a limpeza autônoma em residências, escritórios e outros
espaços fechados onde a manutenção regular é necessária.

In [4]:
# from coppeliasim_zmqremoteapi_client import RemoteAPIClient
import numpy as np

# client = RemoteAPIClient()
# sim = client.require('sim')

## Obtendo o Mapa da Residência

## Wavefront Planner

In [37]:
OCCUPY = np.array([
    [0,0,0,0,0,0,0,0],
    [0,0,0,1,1,1,0,0],
    [0,0,0,1,1,1,0,0],
    [0,0,0,0,0,0,0,0]
])
N = len(OCCUPY)
M = len(OCCUPY[0])

# Get first free cell
FIRST_CELL = None
for i in range(N):
    for j in range(M):
        if OCCUPY[i][j] == 0:
            FIRST_CELL = (i,j)
            break
    if FIRST_CELL != None:
        break

# print(OCCUPY)
# print(N,M)
# print(FIRST_CELL)

INF_VAL = 99999

WAVEFRONT = np.full_like(OCCUPY,INF_VAL)

# Start wavefront algorithm

#          UP   RIGHT DOWN  LEFT 
moves = [(-1,0),(0,1),(1,0),(0,-1)]

def validCell(i,j):
    return 0 <= i < N and 0 <= j < M and OCCUPY[i][j] == 0

def wavefrontPlanner(i0,j0):
    q = []
    q.append((i0,j0,0))
    WAVEFRONT[i0][j0] = 0
    max_k = 0

    while q:
        (i,j,k) = q.pop(0)
        max_k = max(max_k,k)

        # Each cell 4-Connect
        for (di,dj) in moves:
            di += i
            dj += j

            # Out of bounds
            if not validCell(di,dj): continue
            # Already visited
            if WAVEFRONT[di][dj] != INF_VAL: continue

            WAVEFRONT[di][dj] = k + 1
            q.append((di,dj,k+1))
    
    return max_k

K = wavefrontPlanner(FIRST_CELL[0],FIRST_CELL[1])
print(WAVEFRONT)

walle_range = (0,K//2)
print(f"Wall-e:\t({walle_range[0]},{walle_range[1]})")
mo_range = (K//2+1,K)
print(f"Mo :\t({mo_range[0]},{mo_range[1]})")

[[    0     1     2     3     4     5     6     7]
 [    1     2     3 99999 99999 99999     7     8]
 [    2     3     4 99999 99999 99999     8     9]
 [    3     4     5     6     7     8     9    10]]
Wall-e:	(0,5)
Mo :	(6,10)


## Iniciando os Robôs

## Controle para andar entre células

## Simulação